In [1]:
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

In [2]:
def gridsearch(data,true):
    svr = SVR(kernel='rbf')
    Cs = [1,1e1,1e2,1e3,1e4,1e5]
    gammas = [1,1e-1,1e-2,1e-3,1e-4,1e-5]
    parameters = {'C':Cs,'gamma':gammas}

    clf = GridSearchCV(svr, parameters, cv=10, scoring='r2')
    clf.fit(data,true)

    return clf.best_params_ , clf.best_score_

def predict(data,values,test,true,clf):
    svr = clf
    train = svr.fit(data,values)
    a = svr.predict(test)
    df = pd.DataFrame(data=[a,true])
    
    return a,train,df

In [3]:
pd.options.display.float_format = '{:,.2f}'.format

dataset = load_boston()
df = pd.DataFrame(dataset.data)
df.columns = dataset.feature_names
df["PRICE"] = dataset.target

In [4]:
X = df.drop("PRICE", axis=1)
Y = df["PRICE"]
print(X.shape)
print(Y.shape)

(506, 13)
(506,)


In [5]:
scaler = MinMaxScaler().fit(X)
scaled_X = scaler.transform(X)

In [6]:
seed = 1000
test_size = 0.20

X_train, X_test, Y_train, Y_test = train_test_split(scaled_X, Y, test_size = test_size, random_state = seed)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(404, 13)
(102, 13)
(404,)
(102,)


In [7]:
param_a, score_a = gridsearch(scaled_X,Y)

In [8]:
param_a

{'C': 10000.0, 'gamma': 0.01}

In [9]:
# user variables to tune
folds   = 10

In [10]:
# hold different regression models in a single dictionary
models = SVR(C=1e5,gamma=1e-2)

In [11]:
pred,trained,df1 = predict(X_train, Y_train, X_test, Y_test, models)

In [12]:
display(df1)

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
0,22.28,15.48,16.26,43.77,23.91,32.12,11.60,42.70,16.38,16.50,...,24.88,12.74,25.71,20.30,20.96,25.34,20.39,19.78,18.25,21.70
1,21.80,10.20,15.20,43.50,50.00,37.20,7.00,42.30,16.00,19.60,...,24.80,12.30,24.60,24.30,19.00,22.00,20.00,19.90,19.10,21.20


In [13]:
# 10-fold cross validation for each model

k_fold  = KFold(n_splits=folds, random_state=seed)
results = cross_val_score(models, X_train, Y_train, cv=k_fold, scoring='r2')

print("r2:\t",results.mean(),"\nstd:\t",results.std())

r2:	 0.8350537039271366 
std:	 0.11575887581665488


In [14]:
models.score(X_train,Y_train)

0.8941634046285116